# Domain Specific Languages in Lisp on Python

This tutorial is based on a [video](https://www.youtube.com/watch?v=5FlHq_iiDW0) by Rainer Joswig.   In the video, he writes a Lisp solution to an [article](https://www.martinfowler.com/articles/languageWorkbench.html) about Domain Specific Languages by Martin Fowler.    Rainer used Common Lisp - specifically [LispWorks](http://www.lispworks.com), who are our neighbours across the road in the St John's Innovation Centre.    We will translate his solution into [Hy](http://hylang.org), a Lisp which transpiles to Python.

The main points of interest are:
 * [Hy syntax](http://docs.hylang.org/en/stable/tutorial.html#hy-is-a-lisp-flavored-python)
 * [Multiple dispatch (multimethods)](https://en.wikipedia.org/wiki/Multiple_dispatch)
 * [Macros](http://docs.hylang.org/en/stable/tutorial.html#macros)

## Define the data

Let's start by defining the test data.   This is a global variable.   In the Python translation, it will be called `EXAMPLE_DATA`.

In [4]:
(def *example-data*
     "SVCLFOWLER         10101MS0120050313.........................
SVCLHOHPE          10201DX0320050315........................
SVCLTWO           x10301MRP220050329..............................
USGE10301TWO          x50214..7050329...............................")

Let's also define a table of mappings from prefixes to classes and formats.   When translated to Python, this will become a nested Python list called `MAPPINGS`.

In [7]:
(def *mappings*
     [["SVCL" :service-call
              [4 18 :customer-name]
              [19 23 :customer-id]
              [24 27 :call-type-code]
              [28 35 :date-of-call-string]]
      ["USGE" :usage
              [4 8 :customer-id]
              [9 22 :customer-name]
              [30 30 :cycle]
              [31 36 :read-date]]])

## Parsing the SVCL record

Finally, let's declare a class to represent a service call.   In Common Lisp we must define the slots (member variables) but we don't have to do that in Python.    We will give this class a custom `__repr__` method so the toplevel can print it nicely.

In [37]:
(defclass service-call [object]
          (defn --repr-- [self]
                (.format "service-call:\n   {}\n   {}\n   {}\n   {}\n"
                         self.customer-name
                         self.customer-id
                         self.call-type-code
                         self.date-of-call-string)))

Let's define the parser.   In the Common Lisp Object System (CLOS), `defmethod` declares a *multimethod*, which is more powerful than the single dispatch methods provided by Python.   Fortunately Hy has an extension which provides multimethods inspired by Clojure.   We'll import the extension and define a `parse-line-for-class` multimethod which dispatches on the `class` parameter.

In [38]:
(require [hy.contrib.multi [defmulti defmethod default-method]])
(defmulti parse-line-for-class [line class] class)

Here is the parser for a service call.   It creates a new `service-call` object and then iterates over a list of knows field offsets and stores them into slots on the object.   This is a bit more generic than simply making 4 separate `setattr` calls, but the field list is still hard coded. 

In [41]:
(defmethod parse-line-for-class service-call [line class]
      (setv obj (class))
      (for [[start end slot] [[4 18  'customer-name]
                              [19 23 'customer-id]
                              [24 27 'call-type-code]
                              [28 35 'date-of-call-string]]]
           (setattr obj slot (cut line start (+ 1 end))))
      obj)

Let's check that it works.

In [42]:
(parse-line-for-class "SVCLFOWLER         10101MS0120050313........................." service-call)

service-call:
   FOWLER         
   10101
   MS01
   20050313


## Generating the classes automatically

In [50]:
(defmacro defmapping [name typ description]
          `(do 
            (defclass ~name [object])))

<function <lambda> at 0x7f341c3b6a60>

`macroexpand` will print the code which this macro will generate.   We have to quote the argument to `macroexpand`, otherwise it will be expanded by the interpreter.

In [52]:
(macroexpand `(defmapping service-call "SVCL" []))

('do' ('defclass' 'service_call' ['object']))

We can also generate a class for usage reports:

In [53]:
(macroexpand `(defmapping usage "USGE" []))

('do' ('defclass' 'usage' ['object']))

Let's extend `defmapping` to generate `parse-line-for-class` as well.   We can start by dropping in the definition of `parse-line-for-class` from above.

In [54]:
(defmacro defmapping [name typ description]
          `(do 
            (defclass ~name [object])
            (defmethod parse-line-for-class service-call [line class]
                       (setv obj (class))
                       (for [[start end slot] [[4 18  'customer-name]
                                               [19 23 'customer-id]
                                               [24 27 'call-type-code]
                                               [28 35 'date-of-call-string]]]
                            (setattr obj slot (cut line start (+ 1 end))))
                       obj)))

<function <lambda> at 0x7f341c3b6b70>

Our macro now generates a `parse-line-for-class` method along with the class definition.

In [55]:
(macroexpand `(defmapping service-call "SVCL" []))

('do' ('defclass' 'service_call' ['object']) ('defmethod' 'parse_line_for_class' 'service_call' ['line' 'class'] ('setv' 'obj' ('class')) ('for' [['start' 'end' 'slot'] [[4 18 ('quote' 'customer_name')] [19 23 ('quote' 'customer_id')] [24 27 ('quote' 'call_type_code')] [28 35 ('quote' 'date_of_call_string')]]] ('setattr' 'obj' 'slot' ('cut' 'line' 'start' ('+' 1 'end')))) 'obj'))

Unfortunately the `parse-line-for-class` method is hard-coded for `service-call`, so although we can generate a `usage` class, it won't get a suitable `parse-line-for-class` method.

In [56]:
(macroexpand `(defmapping usage "USGE" []))

('do' ('defclass' 'usage' ['object']) ('defmethod' 'parse_line_for_class' 'service_call' ['line' 'class'] ('setv' 'obj' ('class')) ('for' [['start' 'end' 'slot'] [[4 18 ('quote' 'customer_name')] [19 23 ('quote' 'customer_id')] [24 27 ('quote' 'call_type_code')] [28 35 ('quote' 'date_of_call_string')]]] ('setattr' 'obj' 'slot' ('cut' 'line' 'start' ('+' 1 'end')))) 'obj'))

Let's fix that by generating the parsing loop in the macro.   The original method definition already used a loop to parse the entries, so all we need to do is change that loop to generate the appropriate code rather than running directly.

In [57]:
(defmacro defmapping [name typ description]
          `(do
            (defclass ~name [object])
            (defmethod parse-line-for-class ~name [line class]
                       (setv obj (class))
                       (for [[start end slot] ~@description]
                            (setattr obj slot (cut line start (+ 1 end))))
                       obj)))

<function <lambda> at 0x7f341c3b6c80>

This produces the same code for `service-call`

In [58]:
(macroexpand `(defmapping service-call "SVCL" [[4 18 'customer-name]
                                               [19 23 'customer-id]
                                               [24 27 'call-type-code]
                                               [28 35 'date-of-call-string]]))

('do' ('defclass' 'service_call' ['object']) ('defmethod' 'parse_line_for_class' 'service_call' ['line' 'class'] ('setv' 'obj' ('class')) ('for' [['start' 'end' 'slot'] [4 18 ('quote' 'customer_name')] [19 23 ('quote' 'customer_id')] [24 27 ('quote' 'call_type_code')] [28 35 ('quote' 'date_of_call_string')]] ('setattr' 'obj' 'slot' ('cut' 'line' 'start' ('+' 1 'end')))) 'obj'))

When we pass in the slot definitions for `usage`, a different `parse-line-for-class` method is generated.

In [60]:
(macroexpand `(defmapping usage "USGE" [[4 8 'customer-id]
                                        [9 22 'customer-name]
                                        [30 30 'cycle]
                                        [31 36 'read-date]]))

('do' ('defclass' 'usage' ['object']) ('defmethod' 'parse_line_for_class' 'usage' ['line' 'class'] ('setv' 'obj' ('class')) ('for' [['start' 'end' 'slot'] [4 8 ('quote' 'customer_id')] [9 22 ('quote' 'customer_name')] [30 30 ('quote' 'cycle')] [31 36 ('quote' 'read_date')]] ('setattr' 'obj' 'slot' ('cut' 'line' 'start' ('+' 1 'end')))) 'obj'))

## Unrolling the parsing loop

Our current macro generates a loop which iterates over a list of fields at run time.   We can do better than this by unrolling the loop at compile time, so that the generated `parse-line-for-class` method is just straight-line code.   The body of a Lisp function is a list of statements, which we can create by accumulating the statements in our loop.  However a cleaner way to do this is to turn the loop into a list comprehension.

In [63]:
(defmacro defmapping [name typ description]
          `(do
            (defclass ~name [object])
            (defmethod parse-line-for-class ~name [line class]
                       (setv obj (class))
                       ~@(list-comp
                          `(setattr obj ~slot (cut line ~start (+ 1 ~end)))
                          [[start end slot] description])
                       obj)))

<function <lambda> at 0x7f341c3b6bf8>

Now the loop is expanded at compile time and `parse-line-for-class` becomes straight line code.

In [64]:
(macroexpand `(defmapping usage "USGE" [[4 8 'customer-id]
                                        [9 22 'customer-name]
                                        [30 30 'cycle]
                                        [31 36 'read-date]]))

('do' ('defclass' 'usage' ['object']) ('defmethod' 'parse_line_for_class' 'usage' ['line' 'class'] ('setv' 'obj' ('class')) ('setattr' 'obj' ('quote' 'customer_id') ('cut' 'line' 4 ('+' 1 8))) ('setattr' 'obj' ('quote' 'customer_name') ('cut' 'line' 9 ('+' 1 22))) ('setattr' 'obj' ('quote' 'cycle') ('cut' 'line' 30 ('+' 1 30))) ('setattr' 'obj' ('quote' 'read_date') ('cut' 'line' 31 ('+' 1 36))) 'obj'))

## Tying it all together

We can now generate parsers for any number of different record types, but we can't parse a file of records.   We need a helper method which can read the label at the start of the record and dispatch to the appropriate parser.   We will define this method in the `defmapping` macro as well.

In [65]:
(defmulti find-class-for-parser [typ] typ)

In [66]:
(defmacro defmapping [name typ description]
          `(do
            (defclass ~name [object])
            (defmethod parse-line-for-class ~name [line class]
                       (setv obj (class))
                       ~@(list-comp
                          `(setattr obj ~slot (cut line ~start (+ 1 ~end)))
                          [[start end slot] description])
                       obj)
            (defmethod find-class-for-parser ~typ [typ] ~name)))

<function <lambda> at 0x7f341c3b69d8>

We can now finally generate the classes for USGE and SVCL and try them out.

In [68]:
(defmapping service-call "SVCL" [[4 18 'customer-name]
                                 [19 23 'customer-id]
                                 [24 27 'call-type-code]
                                 [28 35 'date-of-call-string]])

(defmapping usage "USGE" [[4 8 'customer-id]
                          [9 22 'customer-name]
                          [30 30 'cycle]
                          [31 36 'read-date]])

Let's try to parse a USGE record.

In [78]:
(parse-line-for-class "USGE10301TWO          x50214..7050329..............................."
                      (find-class-for-parser "USGE"))

We can now iterate over the `*example-data*` which we defined at the beginning, creating an object for each line.

In [79]:
(setv parsed
      (list-comp
       (parse-line-for-class line (find-class-for-parser (cut line 0 4)))
       [line (.split *example-data* "\n")]))

In [80]:
parsed

[<service_call object at 0x7f341c38eef0>, <service_call object at 0x7f341c376978>, <service_call object at 0x7f341c376710>, <usage object at 0x7f341c376e48>]

## Extra

Let's try to get our custom `__repr__` method back

In [98]:
(defclass foobar [object]
          (defn --repr-- [self]
                (.format "service-call:\n   {}\n   {}\n   {}\n   {}\n"
                         self.customer-name
                         self.customer-id
                         self.call-type-code
                         self.date-of-call-string)))

In [100]:
(setv test (foobar))
(print test)

Traceback (most recent call last):
  File "/home/local/seminars/hylang/lib64/python3.6/site-packages/calysto_hy/kernel.py", line 98, in do_execute_direct
    eval(code, self.env)
  File "In [100]", line 2, in <module>
  File "In [98]", line 4, in __repr__
AttributeError: 'foobar' object has no attribute 'customer_name'



In [ ]:
(defmacro defmapping [name typ description]
          `(do
            (defclass ~name [object]
                      (defn --repr-- [self]
                            (.format "service-call:\n   {}\n   {}\n   {}\n   {}\n"
                                     self.customer-name
                                     self.customer-id
                                     self.call-type-code
                                     self.date-of-call-string)))
            (defmethod parse-line-for-class ~name [line class]
                       (setv obj (class))
                       ~@(list-comp
                          `(setattr obj ~slot (cut line ~start (+ 1 ~end)))
                          [[start end slot] description])
                       obj)
            (defmethod find-class-for-parser ~typ [typ] ~name)))